# Configure Database Connection
Set up connection parameters using environment variables or connection string formatting for database credentials.

In [1]:
# Import necessary libraries
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables from main project .env file
load_dotenv(Path.cwd().parent / '.env')

# Get all connection parameters from environment variables
db_ip = os.getenv('DB_IP')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')
db_port = os.getenv('DB_PORT')

# Create the connection string
connection_string = f"postgresql://{db_user}:{db_password}@{db_ip}:{db_port}/{db_name}"

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Test the connection by printing the engine
print(engine)


NameError: name 'db_user' is not defined

# Create SQLAlchemy Engine
Initialize SQLAlchemy engine using the connection string and demonstrate different ways to create the engine.

In [ ]:
# Create SQLAlchemy Engine

# Initialize SQLAlchemy engine using the connection string
engine = create_engine(connection_string)

# Test the connection by printing the engine
print(engine)

# Demonstrate different ways to create the engine
# Using URL object
from sqlalchemy.engine.url import URL

url = URL.create(
    drivername="postgresql",
    username=db_user,
    password=db_password,
    host=db_ip,
    port=db_port,
    database=db_name,
    password=db_password,
    host=db_ip,
    port=db_port,
    database=db_name,
)

engine_from_url = create_engine(url)
print(engine_from_url)

# Using environment variables
os.environ["DB_USER"] = db_user
os.environ["DB_PASSWORD"] = db_password
os.environ["DB_HOST"] = db_ip
os.environ["DB_PORT"] = db_port
os.environ["DB_NAME"] = db_name

connection_string_env = f'postgresql://{os.getenv("DB_USER")}:{os.getenv("DB_PASSWORD")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{os.getenv("DB_NAME")}'
engine_from_env = create_engine(connection_string_env)
print(engine_from_env)


# Establish Connection
Create a database session and demonstrate how to properly manage database connections.

In [ ]:
# Establish Connection

from sqlalchemy.orm import sessionmaker

# Create a configured "Session" class
Session = sessionmaker(bind=engine)

# Create a Session
session = Session()

# Test the connection by executing a simple query
try:
    result = session.execute("SELECT 1")
    print("Connection successful:", result.fetchone())
except Exception as e:
    print("Connection failed:", e)
finally:
    # Close the session
    session.close()


# Execute Basic Query Test
Write and execute a simple SELECT query to verify the connection is working properly.

In [ ]:
# Execute Basic Query Test

from sqlalchemy.orm import sessionmaker

# Create a configured "Session" class
Session = sessionmaker(bind=engine)

# Create a Session
session = Session()

# Write and execute a simple SELECT query to verify the connection is working properly
try:
    result = session.execute("SELECT 1")
    print("Query executed successfully:", result.fetchone())
except Exception as e:
    print("Query execution failed:", e)
finally:
    # Close the session
    session.close()


# Implement Error Handling
Implement try-except blocks for proper error handling during database operations.

In [ ]:
# Implement Error Handling

from sqlalchemy.exc import SQLAlchemyError

# Create a configured "Session" class
Session = sessionmaker(bind=engine)

# Create a Session
session = Session()

# Implement error handling for database operations
try:
    # Attempt to execute a query
    result = session.execute("SELECT 1")
    print("Query executed successfully:", result.fetchone())
except SQLAlchemyError as e:
    # Handle SQLAlchemy-specific errors
    print("SQLAlchemy error occurred:", e)
except Exception as e:
    # Handle general errors
    print("An error occurred:", e)
finally:
    # Ensure the session is closed
    session.close()


# Using Pandas with SQLAlchemy
Demonstrate how to use pandas.read_sql() to fetch query results directly into DataFrames.

In [ ]:
# Using Pandas with SQLAlchemy

import pandas as pd

# Create a configured "Session" class
Session = sessionmaker(bind=engine)

# Create a Session
session = Session()

# Define a query to fetch data
query = "SELECT * FROM your_table_name LIMIT 10"

# Fetch the query results into a DataFrame
try:
    df = pd.read_sql(query, con=engine)
    print("Data fetched successfully:")
    print(df.head())
except SQLAlchemyError as e:
    print("SQLAlchemy error occurred:", e)
except Exception as e:
    print("An error occurred:", e)
finally:
    # Close the session
    session.close()
